In [ ]:
import yfinance as yf
import pandas as pd
from dotenv import load_dotenv
from stocksymbol import StockSymbol
import os
import numpy as np
load_dotenv()

def getHistory(symbol, per):
    stock = yf.Ticker(symbol)
    hist = stock.history(period=per)
    hist["Returns"] = (hist["Close"] / hist["Close"].shift(1)).iloc[1:, ]
    hist["LogReturns"] = np.log(hist["Returns"])
    return hist

def getMarketPrice(symbol):
    ticker = yf.Ticker(symbol).info
    return ticker['regularMarketPreviousClose']

def getCloseReturns(symbol) -> pd.DataFrame:
    price_df = getHistory(symbol, "1mo")
    return (price_df["Close"] / price_df["Close"].shift(1)).iloc[1:, ]

def getLogReturnsFromSymbol(symbol):
    df = getCloseReturns(symbol)
    df["Returns"] = np.log(df)
    return df["Returns"]

# def getLogReturnsHistory(symbol):
#     stock = yf.Ticker(symbol)
#     hist = stock.history(period="1mo")
#     hist["Returns"] = (hist["Close"] / hist["Close"].shift(1)).iloc[1:, ]
#     hist["LogReturns"] = hist["Returns"].apply(lambda x: math.log(x))
#     return hist
    
def getLogReturnsFromList(symbols: list[str]) -> pd.DataFrame:
    df = pd.DataFrame()
    for symbol in symbols:
        df[symbol] = getLogReturnsFromSymbol(symbol)
    return df

ss = StockSymbol(os.environ["STOCK_SYMBOL_API_KEY"])
stock_symbols = list(set(map(lambda x: x['symbol'], ss.get_symbol_list(market="US"))))

if __name__ == '__main__':
    print(stock_symbols)

In [ ]:
import pandas as pd
import inspect

def get_function_arguments(func):
    signature = inspect.signature(func)
    arguments = []
    for name, param in signature.parameters.items():
        if param.default is inspect.Parameter.empty:
            arguments.append((name, None))
        else:
            arguments.append((name, param.default))
    return arguments

def create_data_from_df(df, symbol):
    data = {}
    hist = df
    hist['Date'] = hist.index
    data[symbol] = hist.to_dict('records')
    del hist['Date']
    data['last_date'] = hist.iloc[[-1]].index[0]
    
    return data

def create_df_from_data(data, symbol):
    hist = pd.DataFrame.from_dict(data[symbol])
    hist['Date'] = pd.to_datetime(hist['Date'])
    hist.set_index('Date', inplace=True)
    
    return hist

def get_hist(data, symbol):
    if data is None or (pd.to_datetime(data['last_date']).date() < pd.Timestamp.today('America/New_York').date()) or symbol not in data or 'last_date' not in data or symbol not in data:
        hist = getHistory(symbol, 'max')
        data = create_data_from_df(hist, symbol)
    else:
        hist = create_df_from_data(data, symbol)
    
    return hist, data

In [ ]:
from dash import callback, Output, Input, State, html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
from dash.exceptions import PreventUpdate
import modeling as ml
from datetime import datetime
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

def createInput(name, id, intype):
    return dbc.Stack([
        dbc.Label(name),
        dbc.Input(id, type=intype)
    ], direction='horizontal', gap=3)

def drawSimInputs():
    intype = 'number'
    return dbc.Stack([
        createInput("Simulations", "nsims", intype),
        createInput("Days", "ndays", intype),
    ], direction="horizontal", gap=4)

def make_layout():
    return html.Div([html.H2('Simulations'),
                dbc.Row([
                    dbc.Col(
                        dcc.Graph(id='MCSims'), width='6'
                    ),
                    dbc.Col(
                        dcc.Graph(id='strat'), width='6'
                    )
                ]),
                html.Br(),
                dbc.Row([
                    dbc.Col([
                        dbc.Stack([
                            drawSimInputs(),
                            dcc.Dropdown(options=stock_symbols, id='sim_stock'),
                            dbc.Button('Run Simulations', 'b_sims', color='primary'),
                        ], direction='horizontal', gap=3)
                    ], width=6),
                    dbc.Col([
                        dbc.Stack([
                            dbc.Label(id='sim_prec'),
                            dbc.Label(id='sim_grets')
                        ], direction='horizontal', gap=3),
                    ], width=6)
                ])
            ])

def make_callbacks():
    @callback(
        [
            Output('MCSims', 'figure'),
            Output('strat', 'figure'),
            Output('ticker_data', 'data', allow_duplicate=True),
            Output('sim_data', 'data')
        ],
        [
            Input('b_sims', 'n_clicks'),
            State('nsims', 'value'),
            State('ndays', 'value'),
            State('sim_stock', 'value'),
            State('ticker_data', 'data')
        ],
        background=True,
        running=[
            (Output('b_sims', 'disabled'), True, False),
        ],
        prevent_initial_call=True
    )
    def simulations(b_sims, nsims, ndays, symbol, data):
        
        print (b_sims, nsims, ndays, symbol, data)
        
        if not b_sims or not nsims or not ndays or not symbol:
            raise PreventUpdate
        
        hist, data = get_hist(data, symbol)
        
        res = ml.runSims(
            hist, 
            RandomForestClassifier(random_state=1),
            ['Open', 'Close', 'High', 'Low'],
            ndays,
            nsims
        )
        
        imp = {
            'precision': res['precision'], 
            'grets_avg': res['grets_avg']
        }
        
        return ml.simFigure(hist, res['data']), ml.stratFigure(hist, res, nsims, ndays), dict(data), imp
        
    @callback(
        [
            Output('sim_prec', 'children'),
            Output('sim_grets', 'children')
        ],
        [
            Input('sim_data', 'data')
        ]
    )
    def simulated_precision(data):
        if data is None:
            raise PreventUpdate
        
        score = data['precision']
        rets = data['grets_avg']
        
        return f'Precision: {score:.4f}', f'Gross Returns: {rets:.4f}'

In [ ]:
from dash import Dash

app = Dash(__name__, external_stylesheets=[dbc.themes.SLATE])

app.layout = make_layout()

make_callbacks()

app.run(jupyter_mode="inline")